In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-21-openjdk-amd64/bin/java
%env JVM_PATH=/usr/lib/jvm/java-21-openjdk-amd64/lib/server/libjvm.so

env: JAVA_HOME=/usr/lib/jvm/java-21-openjdk-amd64/bin/java
env: JVM_PATH=/usr/lib/jvm/java-21-openjdk-amd64/lib/server/libjvm.so


## Generate Pseudo Dcos

In [2]:
import requests
# from pyserini.search import get_topics, get_qrels
from tqdm import tqdm
import json

url = "http://localhost:8000/generate"
topic_name = "dl19-passage"

# topics = get_topics(topic_name)
with open("dl19-passage_topic_info.json", "r") as f:
    topics = json.load(f)

gen_config = {
    "max_new_tokens": 128,
    "do_sample": False
}

gen_pseudo_docs = {"gen_config": gen_config}

for topic in tqdm(topics[:1], desc=f"Generating pseudo-docs for {topic_name} with probing probability included"):
    query = topic["query"]
    prompt = f"""Please write a passage to answer the question
Question: {query}
Passage:"""

    response = requests.post(url, json={"message": prompt, "gen_config": gen_config}).json()
    
    print(response)

Generating pseudo-docs for dl19-passage with probing probability included: 100%|██████████| 1/1 [00:05<00:00,  5.36s/it]

{'generated_text': " Fleas are ectoparasites that feed on the blood of mammals and birds. The life cycle of a flea consists of four stages: egg, larva, pupa, and adult. The female flea lays her eggs on the host animal, which hatch into larvae after 2-3 weeks. The larvae feed on organic matter and debris, and then spin a cocoon to enter the pupal stage. The pupal stage lasts for 2-4 weeks, during which the flea develops into an adult. The adult flea emerges from the cocoon and begins to feed on the host animal's blood. The entire life cycle of", 'word_predictions': [{'word': 'Fleas', 'predicted_probability': 0.9943206310272217, 'predicted_label': 1}, {'word': 'are', 'predicted_probability': 0.9997876286506653, 'predicted_label': 1}, {'word': 'ectoparasites', 'predicted_probability': 0.993155300617218, 'predicted_label': 1}, {'word': 'that', 'predicted_probability': 0.9999582767486572, 'predicted_label': 1}, {'word': 'feed', 'predicted_probability': 0.9996243715286255, 'predicted_label':

In [2]:
response['generated_text']

"Please write a passage to answer the question\nQuestion: how long is life cycle of flea\nPassage: Fleas are ectoparasites that feed on the blood of mammals and birds. The life cycle of a flea consists of four stages: egg, larva, pupa, and adult. The female flea lays her eggs on the host animal, which hatch into larvae after 2-3 weeks. The larvae feed on organic matter and debris, and then spin a cocoon to enter the pupal stage. The pupal stage lasts for 2-4 weeks, during which the flea develops into an adult. The adult flea emerges from the cocoon and begins to feed on the host animal's blood. The entire life cycle of"

In [5]:
from pyserini.search import get_topics, get_qrels

topics = get_topics("msmarco-passage-dev-subset")
qrels = get_qrels("msmarco-passage-dev-subset")

In [7]:
print(len(topics))
print(len(qrels))

6980
6980


In [27]:
for topic in topics:
    print(topic, topics[topic])
    break

1102330 {'title': 'why do people grind teeth in sleep'}


In [29]:
for qid in qrels:
    print(qid, qrels[qid])
    print(1102330, qrels[1102330])
    break

300674 {7067032: '1'}
1102330 {7867438: '1'}


In [24]:
relevant_doc_cnt = []
for qid in qrels:
    relevant_doc_cnt.append(len(qrels[qid].keys()))

# descriptive of relevant documents
print(f"max relevant docs: {max(relevant_doc_cnt)}")


max relevant docs: 4


Fetching Document Content using Document ID (With Pyserini)

https://github.com/castorini/pyserini/blob/master/docs/usage-fetch.md

In [30]:
from pyserini.search.lucene import LuceneSearcher
searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')
doc = searcher.doc('7067032')

import json
json_doc = json.loads(doc.raw())

json_doc['contents']

'http://en.wikipedia.org/wiki/William_Bradford_(Plymouth_Colony_governor) William Bradford (c.1590 â\x80\x93 1657) was an English Separatist leader in Leiden, Holland and in Plymouth Colony was a signatory to the Mayflower Compact. He served as Plymouth Colony Governor five times covering about thirty years between 1621 and 1657.'

In [41]:
from pyserini.search import get_topics, get_qrels
from pyserini.search.lucene import LuceneSearcher
import json
    
collection = "msmarco-passage-dev-subset"

searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')

topics = get_topics(collection)
qrels = get_qrels(collection)

q_and_its_rel_docs = {}

for qid in topics:
    query = topics[qid]['title']
    q_and_its_rel_docs[qid] = {}
    q_and_its_rel_docs[qid]['qid'] = qid
    q_and_its_rel_docs[qid]['query'] = query
    q_and_its_rel_docs[qid]['rel_docs'] = []

    rel_docs = qrels[qid]
    for docid in rel_docs:
        rank = rel_docs[docid]
        doc = searcher.doc(str(docid)) # str means using external collection id
        json_doc = json.loads(doc.raw())
        content = json_doc['contents']
        q_and_its_rel_docs[qid]['rel_docs'].append({
            "docid": docid,
            "content": content,
            "rank": rank
        })

with open(f"{collection}_q_and_its_rel_docs.json", "w") as f:
    json.dump(q_and_its_rel_docs, f)

In [39]:
collection = "dl19-passage"

searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')

topics = get_topics(collection)
qrels = get_qrels(collection)

print(list(qrels.items())[0])

(19335, {1017759: '0', 1082489: '0', 109063: '0', 1160863: '0', 1160871: '0', 1189088: '0', 1203500: '0', 1231806: '0', 1231807: '0', 1274615: '0', 1274620: '0', 1324075: '0', 1509459: '0', 1555317: '0', 1568085: '0', 161603: '0', 1705525: '0', 1720387: '0', 1720388: '0', 1720389: '1', 1720393: '0', 1720395: '1', 1722: '0', 1725697: '0', 1726: '0', 1729: '2', 1730: '0', 1796642: '0', 1796647: '0', 1825416: '0', 1825418: '0', 1837110: '0', 1871222: '0', 1908804: '0', 1956669: '0', 1958100: '0', 1958102: '0', 1958103: '0', 1959553: '0', 2004186: '0', 2046505: '1', 2071723: '0', 2130187: '0', 2186129: '0', 2304004: '0', 2304005: '0', 2324839: '0', 2325143: '0', 2382766: '0', 2394677: '0', 256744: '0', 256746: '0', 256750: '0', 2594897: '0', 2604487: '0', 2725017: '0', 2874503: '0', 2943092: '0', 2978577: '0', 3045565: '1', 3045567: '1', 3137952: '0', 3175481: '3', 3175483: '0', 3175484: '3', 3175485: '0', 3212083: '0', 3260688: '0', 3358752: '0', 342431: '0', 342432: '0', 3424644: '0', 34

## Load Pseudo Docs with word probability
Design different strategies to utilize word probability to refine the pseudo documents.

### Strategy 1 (word-level): Remove unconfident words
For each word labeled with 0 (unconfident), we can remove it from the pseudo document.

### Strategy 2: (statement-level): Remove unconfident statements
1. Divide the pseudo document into multiple statements (split by period or comma)
2. Check the last word probability of each statement. If the last word is labeled with 0, we can remove the statement from the pseudo document.

### Strategy 3: (statement-level): Thresholding on the last word probability to repeat confident statements more times
1. Divide the pseudo document into multiple statements (split by period or comma).
2. Check the last word probability of each statement. 
3. Set multiple thresholds for the last word probability. Statements with last word probability in the maximum threshold will be repeated more times in the pseudo document.

In [48]:
class ApplyProbingStrategy:
    def apply(self, passage: str, word_prob: list[dict]):
        raise NotImplementedError("Subclasses should implement this method")

class UseOrigianlPassage(ApplyProbingStrategy):
    def apply(self, passage: str, word_prob: list[dict]):
        return passage    
class RemoveUnconfidentWord(ApplyProbingStrategy):
    def __init__(self, should_check_last_word_confidence = False):
        self.should_check_last_word_confidence = should_check_last_word_confidence
    
    def direct_remove(self, word_prob: list[dict]):
        for word_info in word_prob:
            if word_info['predicted_label'] == 1:
                new_passage += word_info['word'] + " "
    def apply(self, passage: str, word_prob: list[dict]):
        if not self.should_check_last_word_confidence:
            return self.direct_remove(word_prob)
        
        new_passage = []
        words_and_label_in_a_statement = []  # list of tuple: (word, label)

        for word_info in word_prob:
            word = word_info['word']
            label = word_info['predicted_label']
            
            if word.endswith("."):
                if label == 1:
                    new_passage.extend(word for word, label in words_and_label_in_a_statement)
                else:
                    new_passage.extend(word for word, label in words_and_label_in_a_statement if label == 1)
                words_and_label_in_a_statement = []
            else:
                words_and_label_in_a_statement.append((word, label))

        return " ".join(new_passage)


class RepeatConfidentWord(ApplyProbingStrategy):
    def __init__(self, three_times_threshold = 0.9, two_times_threshold = 0.6):
        self.three_times_threshold = three_times_threshold
        self.two_times_threshold = two_times_threshold
    def determine_repeat_times(self, word, predicted_probability):
        if predicted_probability >= self.three_times_threshold:
            return " ".join([word] * 3)
        elif predicted_probability >= self.two_times_threshold:
            return " ".join([word] * 2)
        else:
            return word
    def apply(self, passage: str, word_prob: list[dict]):
        new_passage = ""
        for word_info in word_prob:
            new_passage += self.determine_repeat_times(word_info['word'], word_info['predicted_probability']) + " "
        return new_passage

class RemoveUnconfidentStatement(ApplyProbingStrategy):
    def apply(self, passage: str, word_prob: list[dict]):
        new_passage = ""
        cur_statement = ""
        for word_info in word_prob:
            cur_statement += word_info['word'] + " "
            if len(word_info['word']) >= 1 and word_info['word'][-1] == ".":
                if word_info['predicted_label'] == 1:
                    new_passage += cur_statement
                cur_statement = ""
        if len(cur_statement) > 0:
            new_passage += cur_statement # if the last word doesn't end with ".", add it to the new passage
        return new_passage
    
class RepeatMoreForConfidentStatement(ApplyProbingStrategy):
    def __init__(self, three_times_threshold = 0.9, two_times_threshold = 0.6):
        self.three_times_threshold = three_times_threshold
        self.two_times_threshold = two_times_threshold
    def determine_repeat_times(self, statement: str, predicted_probability: float):
        if predicted_probability >= self.three_times_threshold:
            return ". ".join([statement] * 3) + "."
        elif predicted_probability >= self.two_times_threshold:
            return ". ".join([statement] * 2) + "."
        else:
            return statement
    def apply(self, passage: str, word_prob: list[dict]):
        new_passage = ""
        cur_statement = ""
        for word_info in word_prob:
            cur_statement += word_info['word'] + " "
            if len(word_info['word']) >= 1 and word_info['word'][-1] == ".":
                new_passage += self.determine_repeat_times(cur_statement, word_info['predicted_probability'])
                cur_statement = ""
        if len(cur_statement) > 0:
            new_passage += self.determine_repeat_times(cur_statement, word_prob[-1]['predicted_probability'])
        return new_passage

In [70]:
from pyserini.search import FaissSearcher, LuceneSearcher
from pyserini.search.faiss import AutoQueryEncoder

class Searcher:
    def __init__(self, index_path: str):
        self.index_path = index_path
        self.searcher = None
    def search(self, query: str, k: int):
        raise NotImplementedError("Subclasses should implement this method")

class BM25Searcher(Searcher):
    def __init__(self, index_path: str):
        super().__init__(index_path)
        self.searcher = LuceneSearcher.from_prebuilt_index(index_path)
    def search(self, query: str, k: int):
        return self.searcher.search(query, k=k)

class ContrieverSearcher(Searcher):
    def __init__(self, index_path: str, encode_dir: str):
        super().__init__(index_path)
        self.encoder = AutoQueryEncoder(encoder_dir=encode_dir, pooling='mean')
        self.searcher = FaissSearcher(index_path, self.encoder)
    def search(self, query: str, k: int):
        return self.searcher.search(query, k=k)


In [66]:
import json
from pyserini.search import LuceneSearcher
from tqdm import tqdm
import subprocess

def apply_strategy_and_eval(
          probing_apply_strategy: ApplyProbingStrategy, 
          pseudo_doc_file: str, 
          output_file: str,
          searcher: Searcher = BM25Searcher('msmarco-v1-passage'), 
    ):     
    with open(pseudo_doc_file, "r") as f:
        data = json.load(f)

    pseudo_docs = data['topics']

    with open(output_file, "w") as f:
        for doc in tqdm(pseudo_docs, desc=f"Perform Searching"):
                qid = doc['qid']
                query = doc['query']
                passage = doc['passage']
                word_prob = doc['word_predictions']
                new_passage = probing_apply_strategy.apply(passage, word_prob)
                hits = searcher.search(query=f"{query*4} {new_passage}", k=1000)
                rank = 0 
                for hit in hits:
                    rank += 1
                    f.write(f'{qid} Q0 {hit.docid} {rank} {hit.score} rank\n')
    commands = [
        f"python -m pyserini.eval.trec_eval -c -l 2 -m map dl19-passage {output_file}",
        f"python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 dl19-passage {output_file}",
        f"python -m pyserini.eval.trec_eval -c -l 2 -m recall.1000 dl19-passage {output_file}"
    ]
    print("Evaluating...")
    for command in commands:
        subprocess.run(command, shell=True)

Query2Doc

In [43]:
pseudo_doc_file = "pseudo-docs/dl19-passage/1.json"
output_file = "pseudo-docs/dl19-passage/eval/bm25-4q-passage-query2doc.trec"
apply_strategy_and_eval(UseOrigianlPassage(), pseudo_doc_file, output_file)

Perform Searching: 100%|██████████| 43/43 [00:09<00:00,  4.40it/s]


Evaluating...
map                   	all	0.3898
ndcg_cut_10           	all	0.5984
recall_1000           	all	0.8445


Strategy1: RemoveUnconfidentWord

In [44]:
pseudo_doc_file = "pseudo-docs/dl19-passage/1.json"
output_file = "pseudo-docs/dl19-passage/eval/bm25-4q-passage-strategy1.trec"
apply_strategy_and_eval(RemoveUnconfidentWord(), pseudo_doc_file, output_file)

Perform Searching:   0%|          | 0/43 [00:00<?, ?it/s]

Perform Searching: 100%|██████████| 43/43 [00:07<00:00,  5.59it/s]


Evaluating...
map                   	all	0.3764
ndcg_cut_10           	all	0.5918
recall_1000           	all	0.8325


Strategy2: RepeatConfidentWord

In [42]:
pseudo_doc_file = "pseudo-docs/dl19-passage/1.json"
output_file = "pseudo-docs/dl19-passage/eval/bm25-4q-passage-RepeatConfidentWord.trec"
apply_strategy_and_eval(RepeatConfidentWord(), pseudo_doc_file, output_file)

Perform Searching: 100%|██████████| 43/43 [00:09<00:00,  4.69it/s]


Evaluating...
map                   	all	0.3353
ndcg_cut_10           	all	0.5277
recall_1000           	all	0.7974


In [46]:
pseudo_doc_file = "pseudo-docs/dl19-passage/1.json"
output_file = "pseudo-docs/dl19-passage/eval/bm25-4q-passage-RepeatConfidentWord-1-0.9.trec"
apply_strategy_and_eval(RepeatConfidentWord(1, 0.9), pseudo_doc_file, output_file)

Perform Searching: 100%|██████████| 43/43 [00:09<00:00,  4.61it/s]


Evaluating...
map                   	all	0.3573
ndcg_cut_10           	all	0.5448
recall_1000           	all	0.8201


In [47]:
pseudo_doc_file = "pseudo-docs/dl19-passage/1.json"
output_file = "pseudo-docs/dl19-passage/eval/bm25-4q-passage-RepeatConfidentWord-0.9-0.8.trec"
apply_strategy_and_eval(RepeatConfidentWord(0.9, 0.8), pseudo_doc_file, output_file)

Perform Searching: 100%|██████████| 43/43 [00:08<00:00,  4.83it/s]


Evaluating...
map                   	all	0.3289
ndcg_cut_10           	all	0.5236
recall_1000           	all	0.7877


In [49]:
pseudo_doc_file = "pseudo-docs/dl19-passage/1.json"
output_file = "pseudo-docs/dl19-passage/eval/bm25-4q-passage-RemoveUnconfidentStatement.trec"
apply_strategy_and_eval(RemoveUnconfidentStatement(), pseudo_doc_file, output_file)

Perform Searching: 100%|██████████| 43/43 [00:07<00:00,  5.39it/s]


Evaluating...
map                   	all	0.3718
ndcg_cut_10           	all	0.5956
recall_1000           	all	0.8371


In [50]:
pseudo_doc_file = "pseudo-docs/dl19-passage/1.json"
output_file = "pseudo-docs/dl19-passage/eval/bm25-4q-passage-RepeatMoreForConfidentStatement.trec"
apply_strategy_and_eval(RepeatMoreForConfidentStatement(), pseudo_doc_file, output_file)

Perform Searching: 100%|██████████| 43/43 [00:09<00:00,  4.68it/s]


Evaluating...
map                   	all	0.3567
ndcg_cut_10           	all	0.5332
recall_1000           	all	0.8289


In [51]:
pseudo_doc_file = "pseudo-docs/dl19-passage/1.json"
output_file = "pseudo-docs/dl19-passage/eval/bm25-4q-passage-RepeatMoreForConfidentStatement-1-0.9.trec"
apply_strategy_and_eval(RepeatMoreForConfidentStatement(1,0.9), pseudo_doc_file, output_file)

Perform Searching: 100%|██████████| 43/43 [00:09<00:00,  4.52it/s]


Evaluating...
map                   	all	0.3800
ndcg_cut_10           	all	0.5704
recall_1000           	all	0.8431


In [52]:
pseudo_doc_file = "pseudo-docs/dl19-passage/1.json"
output_file = "pseudo-docs/dl19-passage/eval/bm25-4q-passage-RepeatMoreForConfidentStatement-1-0.95.trec"
apply_strategy_and_eval(RepeatMoreForConfidentStatement(1,0.95), pseudo_doc_file, output_file)

Perform Searching: 100%|██████████| 43/43 [00:09<00:00,  4.58it/s]


Evaluating...
map                   	all	0.3798
ndcg_cut_10           	all	0.5761
recall_1000           	all	0.8453


## Contriever

In [68]:
searcher = ContrieverSearcher('contriever_msmarco_index/', 'facebook/contriever')

In [69]:
pseudo_doc_file = "pseudo-docs/dl19-passage/1.json"
output_file = "pseudo-docs/dl19-passage/eval/contriever-4q-passage-RepeatMoreForConfidentStatement-1-0.9.trec"
apply_strategy_and_eval(RepeatMoreForConfidentStatement(1,0.9), pseudo_doc_file, output_file, searcher)

Perform Searching: 100%|██████████| 43/43 [02:02<00:00,  2.85s/it]


Evaluating...
map                   	all	0.3418
ndcg_cut_10           	all	0.5418
recall_1000           	all	0.8139


In [ ]:
pseudo_doc_file = "pseudo-docs/dl19-passage/1.json"
output_file = "pseudo-docs/dl19-passage/eval/contriever-4q-passage-UseOrigianlPassage.trec"
apply_strategy_and_eval(UseOrigianlPassage(), pseudo_doc_file, output_file, searcher)